In [2]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [59]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

class CIFAR10_CNN(nn.Module):
  def __init__(self, config):
    super(CIFAR10_CNN, self).__init__()

    # 첫번째 층: Convolutional NN
    # (batch, 32, 32, 3) -> (batch, 32, 32, 9) -> (batch, 16, 16, 9)
    self.conv1 = nn.Sequential()
    self.conv1.add_module('conv1', nn.Conv2d(3, 9, kernel_size=(3, 3), stride = (1, 1), padding = (1, 1)))
    self.conv1.add_module('relu', nn.ReLU())
    self.conv1.add_module('maxpool1', nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)))

    # 두번째 층: Convolutional NN
    # (batch, 16, 16, 9) -> (batch, 16, 16, 72) -> (batch, 8, 8, 72)
    self.conv2 = nn.Sequential(
        nn.Conv2d(9, 72, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2))
    
    # 네번째 층: Fully-Connected NN
    # (batch, 8, 8, 72) -> (batch, 10)
    self.fnn = nn.Linear(8*8*72, 10, bias = True)

    # FNN 가중치 초기화
    nn.init.xavier_uniform_(self.fnn.weight)

  def forward(self, input_features):

    # Convolution
    output = self.conv1(input_features)
    output = self.conv2(output)

    # 텐서를 1차원으로 펼치기: (batch, -1)
    # output.size(0): 배치 차원의 크기, -1: 해당 차원은 파이토치가 알아서 설정
    output = output.view(output.size(0), -1)
    hypothesis = self.fnn(output)
   
    return hypothesis

In [60]:
# batch file to dict
def batch2dict(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# 데이터 읽기
def load_dataset():
  batch_1 = "/gdrive/MyDrive/ml_colab/week10/cifar-10-batches-py/data_batch_1"
  batch_2 = "/gdrive/MyDrive/ml_colab/week10/cifar-10-batches-py/data_batch_2"
  batch_3 = "/gdrive/MyDrive/ml_colab/week10/cifar-10-batches-py/data_batch_3"
  batch_4 = "/gdrive/MyDrive/ml_colab/week10/cifar-10-batches-py/data_batch_4"
  batch_5 = "/gdrive/MyDrive/ml_colab/week10/cifar-10-batches-py/data_batch_5"
  test_batch = "/gdrive/MyDrive/ml_colab/week10/cifar-10-batches-py/test_batch"

  batch_1_data = batch2dict(batch_1)[b'data'].reshape(-1,3,32,32)
  batch_2_data = batch2dict(batch_2)[b'data'].reshape(-1,3,32,32)
  batch_3_data = batch2dict(batch_3)[b'data'].reshape(-1,3,32,32)
  batch_4_data = batch2dict(batch_4)[b'data'].reshape(-1,3,32,32)
  batch_5_data = batch2dict(batch_5)[b'data'].reshape(-1,3,32,32)
  test_data = batch2dict(test_batch)[b'data'].reshape(-1,3,32,32)

  batch_1_labels = batch2dict(batch_1)[b'labels']
  batch_2_labels = batch2dict(batch_2)[b'labels']
  batch_3_labels = batch2dict(batch_3)[b'labels']
  batch_4_labels = batch2dict(batch_4)[b'labels']
  batch_5_labels = batch2dict(batch_5)[b'labels']
  test_labels = batch2dict(test_batch)[b'labels']

  batch_1_labels.extend(batch_2_labels)
  batch_1_labels.extend(batch_3_labels)
  batch_1_labels.extend(batch_4_labels)
  batch_1_labels.extend(batch_5_labels)

  train_X = np.vstack((batch_1_data, batch_2_data, batch_3_data, batch_4_data, batch_5_data))
  train_y = batch_1_labels
  test_X = test_data
  test_y = test_labels
  print(train_X.shape) # (50000, 3, 32, 32)
  print(test_X.shape) # (10000, 3, 32, 32)

  train_X = torch.tensor(train_X, dtype=torch.float)
  train_y = torch.tensor(train_y, dtype=torch.long)
  test_X = torch.tensor(test_X, dtype=torch.float)
  test_y = torch.tensor(test_y, dtype=torch.long)
  
  return (train_X, train_y), (test_X, test_y)

In [61]:
# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

# 평가 수행 함수
def do_test(model, test_dataloader):

  # 평가 모드 setting
  model.eval()

  # batch 별로 예측값과 정답을 저장할 리스트 초기화
  predicts, golds = [], []
  
  with torch.no_grad():
    for step, batch in enumerate(test_dataloader):
  
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      input_features, labels = batch
      hypothesis = model(input_features)

      # ont-hot 표현으로 변경
      logits = torch.argmax(hypothesis, -1)

      x = tensor2list(logits)
      y = tensor2list(labels)

      # 예측값과 정답을 리스트에 추가
      predicts.extend(x)
      golds.extend(y)
    
    print("PRED=",predicts)
    print("GOLD=",golds)
    print("Accuracy= {0:f}\n".format(accuracy_score(golds, predicts)))

# 모델 평가 함수
def test(config):
  model = CIFAR10_CNN(config).cuda()

  # 저장된 모델 가중치 로드
  model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])))

  # 데이터 load
  (_, _), (features, labels) = load_dataset()
  
  test_features = TensorDataset(features, labels)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])
  
  do_test(model, test_dataloader)

In [62]:
# 모델 학습 함수
def train(config):
  # 모델 생성
  model = CIFAR10_CNN(config).cuda()

  # 데이터 읽기
  (input_features, labels), (_, _) = load_dataset()

  # TensorDataset/DataLoader를 통해 배치(batch) 단위로 데이터를 나누고 셔플(shuffle)
  train_features = TensorDataset(input_features, labels)
  train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

  # 크로스엔트로피 비용 함수 
  loss_func = nn.CrossEntropyLoss()
  # 옵티마이저 함수 (역전파 알고리즘을 수행할 함수)
  optimizer = torch.optim.Adam(model.parameters(), lr=config["learn_rate"])

  for epoch in range(config["epoch"]+1):
    # 학습 모드 setting
    model.train()
  
    # epoch 마다 평균 비용을 저장하기 위한 리스트
    costs = []

    for (step, batch) in enumerate(train_dataloader):

      # batch = (input_features[step], labels[step])*batch_size
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      # 각 feature 저장
      input_features, labels = batch

      # 역전파 변화도 초기화
      # .backward() 호출 시, 변화도 버퍼에 데이터가 계속 누적한 것을 초기화
      optimizer.zero_grad()

      # H(X) 계산: forward 연산
      hypothesis = model(input_features)
      # 비용 계산
      cost = loss_func(hypothesis, labels)
      # 역전파 수행
      cost.backward()
      optimizer.step()
   
      # 현재 batch의 스텝 별 loss 저장
      costs.append(cost.data.item())
    
    # 에폭마다 평균 비용 출력하고 모델을 저장
    print("epoch {0} - Average Loss= {1:f}".format(epoch, np.mean(costs)))
    torch.save(model.state_dict(), os.path.join(config["output_dir"], "epoch_{0:d}.pt".format(epoch)))
    do_test(model, train_dataloader)

In [63]:
if(__name__=="__main__"):

    root_dir = "/gdrive/My Drive/ml_colab/week10/cifar10"
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "model_name":"epoch_{0:d}.pt".format(10),
              "output_dir":output_dir,
              "learn_rate":0.001,
              "batch_size":32,
              "epoch":20,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

Output hidden; open in https://colab.research.google.com to view.